In [ ]:
import requests
import pandas as pd
import json
import math
# Define your API key and endpoint
ENDPOINT = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json'

#csv to pandas
df = pd.read_csv('orenda_Kyiv_lun_rieltor.csv')

test_df = df.head(10)

#iterate over dataframe
for index, row in df.iterrows():
    latitude = row.latitude
    longitude = row.longitude

    params = {
        'key': API_KEY,
        'location': f'{latitude},{longitude}',
        'radius': 5000,
        'type': 'subway_station'
    }

    # Send the nearby search request
    response = requests.get(ENDPOINT, params=params)
    results = response.json()

    json_string = json.dumps(results, indent=4)

    print(json_string)

    # if results not empty
    if len(results['results']) > 0:
        # Get the closest subway station
        closest_station = None
        closest_distance = float('inf')  # Set initial distance to infinity

        for subway_station in results['results']:
            location = subway_station['geometry']['location']
            lat = location['lat']
            lng = location['lng']

            # Calculate distance between given location and current subway station
            d_lat = math.radians(lat - latitude)
            d_lng = math.radians(lng - longitude)
            a = math.sin(d_lat / 2) ** 2 + math.cos(math.radians(latitude)) * math.cos(math.radians(lat)) * math.sin(d_lng / 2) ** 2
            c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
            distance = 6371 * c * 1000  # 6371 km is the approximate radius of the Earth, multiplied by 1000 to convert to meters

            # Update closest station if current station is closer
            if distance < closest_distance:
                closest_station = subway_station
                closest_distance = distance

        if closest_station:
            name = closest_station['name']
            l = closest_station['geometry']['location']
            location = f'{l["lat"]}, {l["lng"]}'
            df.at[index, "Closest_subway"] = name
            df.at[index, "Closest_subway_location"] = location

            df.to_csv('rieltor_subways.csv', index=False)
        else:
            print('No subway stations found within the specified radius.')

In [60]:
import googlemaps

gmaps = googlemaps.Client(key=API_KEY)
df = pd.read_csv('rieltor_subways.csv')

for index, row in df.iterrows():
    lat = row.latitude
    lng = row.longitude

    if pd.isna(row.Closest_subway_location):
        continue

    loc = row.Closest_subway_location.split(',')
    loc = [float(i) for i in loc]

    a = (lat, lng)  # Latitude and longitude of point A
    b = (loc[0], loc[1])  # Latitude and longitude of point B

    result = gmaps.distance_matrix(a, b, mode='walking', units='metric')
    r = json.dumps(result, indent=4)

    df.at[index, "Distance Text"] = result['rows'][0]['elements'][0]['distance']['text']
    df.at[index, "Distance Value"] = result['rows'][0]['elements'][0]['distance']['value']
    df.at[index, "Duration Text"] = result['rows'][0]['elements'][0]['duration']['text']
    df.at[index, "Duration Value"] = result['rows'][0]['elements'][0]['duration']['value']

    df.to_csv('rieltor_subways.csv', index=False)


In [62]:
#change final data to be easier to test

df['latitude'] = df['latitude'].astype(str)
df['longitude'] = df['longitude'].astype(str)

# Concatenate latitude and longitude with comma separator
df['target_location'] = df['latitude'] + ',' + df['longitude']

# Drop original latitude and longitude columns
df.drop(['latitude', 'longitude'], axis=1, inplace=True)

df.to_csv('rieltor_subways_formatted.csv', index=False)

In [69]:
# format distance

df = pd.read_csv('rieltor_subways_formatted.csv')

df['Distance to subway in (km)'] = df['Distance Text'].str.replace(' km', '')
df.drop(['Distance Text'], axis=1, inplace=True)

#rename column name
df.rename(columns={'Distance Value': 'Distance to subway in (m)'}, inplace=True)
df.rename(columns={'Duration Value': 'Distance to subway in seconds'}, inplace=True)

hours = df['Duration Text'].str.extract(r'(\d+) hour')
minutes = df['Duration Text'].str.extract(r'(\d+) min')

# Convert to numeric values and fill missing values with 0
hours = hours.astype(float).fillna(0)
minutes = minutes.astype(float).fillna(0)

# Convert hours to minutes and add to minutes
total_minutes = hours * 60 + minutes

# Replace the original column with the total minutes
df['Duration to subway in mins'] = total_minutes

df.rename(columns={'Duration Text': 'Time to walk to subway'}, inplace=True)

df.to_csv('rieltor_subways_formatted_2.csv', index=False)